# Tutorial - QFrame

## What is a QFrame?
QFrame is a class which generates an SQL statement. It stores fields info in `QFrame.data` parameter which is a dictionary.

`QFrame.data` has `select` key in which it stores `fields` which we want to have in our SQL statement. Each key have to have specified `type` which can be 'dim' if the varibale is a dimension variable or 'num' if the variable is a numeric variable. Let's take a look at all options that we can have under `select` and `fields` keys.

```json
{
  "select": {
    "table": "table",
    "schema": "schema",
    "fields": {
      "column": {
        "type": "dim",
        "as": "",
        "group_by": "",
        "order_by": "",
        "expression": "",
        "select": "",
        "custom_type": ""
      }
    },
    "where": "",
    "distinct": "",
    "having": "",
    "limit": ""
  }
}
```

- `table` - Name of the table.
- `schema` - Name of the schema.
- `fields`, in each field:
    - `type` - Type of the column. Options:

        - 'dim' - VARCHAR(500)  
        - 'num' - FLOAT
     
     Every column has to have specified type. If you want to sepcify another type check `custom_type`.
    - `as` - Column alias (name).

    - `group_by` - Aggregation type. Possibilities:

        - 'group' - This field will go to GROUP BY statement.
        - {'sum', 'count', 'min', 'max', 'avg'} - This field will by aggregated in specified way.
  
     If you don't want to aggregate fields leave `group_by` empty in each field.
    - `order_by` - Put the field in order by statement. Options:
    
        - 'ASC'
        - 'DESC'
        
    - `expression` - Expression, eg. CASE statement, column operation, CONCAT statement, ... .
    - `select` - Set 0 if you don't want to put this field in SELECT statement.
    - `custom_type` - Specify custom SQL data type, eg. DATE.
- `where` - Add where statement, eg. 'sales>100'
- `distinct` - Set 1 to add distinct to select
- `having` - Add having statement, eg. 'sum(sales)>100'
- `limit` - Add limit, eg. 100

## How to create a QFrame?
You can create a QFrame manually - passing the data directly to QFrame or automatically - using `QFrame.from_table()` method. Before you generate a QFrame you have to set up your ODBC. If you set up `dsn` (data source name) parameter in grizly configuration file then when you create a QFrame you only need to pass `dsn`. In other case you should also specify `dialect` and `db` parameters.

In [1]:
from grizly import (
    get_path, 
    QFrame
)

### Manually - using dictionary

This method is the most direct method of creating a QFrame - to use it you need to know the structure of `QFrame.data`. From following dictionary

In [2]:
data = {
  "select": {
    "table": "table",
    "schema": "schema",
    "fields": {
      "col": {
        "type": "dim"
      }
    }
  }
}

QFrame will generate a simple sql

In [3]:
qf = QFrame(dsn="redshift_acoe").from_dict(data)
print(qf)

SELECT col
FROM schema.table


Here we also used simple method `.get_sql()` which prints sql saved in QFrame.

### Manually - using JSON file

We use a `.json` file to conviniently manipulate information about columns, renames and other things that might be very verbose to manipulate in python code. We can edit the json file into a json editor like http://jsoneditoronline.org/ more conviniently than in Python code.

After editing the `store.json` we can read it back inside a QFrame using `from_json()`.

This means we can use our json as our main `store` of verbose information and python as our main way to manipulate said information.

In [4]:
json_path = get_path("grizly_dev", "notebooks", "store.json")
qf.save_json(json_path=json_path, subquery="my_query_1")

qf = QFrame(dsn="redshift_acoe").from_json(json_path=json_path, subquery="my_query_1")
print(qf)

SELECT col
FROM schema.table


### Automatically - using from_table method

The other way to generate a QFrame is to use `QFrame.from_table()` method.

In [5]:
qf = QFrame(dsn="redshift_acoe")
qf.from_table(schema="administration", table="table_tutorial")

print(qf)

SELECT col1,
       col2,
       col3,
       col4
FROM administration.table_tutorial


In [6]:
qf_c = QFrame(dsn="redshift_acoe")
qf_c.from_table(schema="administration", table="table_tutorial", columns=["col1", "col3"])

print(qf_c)

SELECT col1,
       col3
FROM administration.table_tutorial


You can also specify which columns you want to pull from database by using `columns` parameter.

## Working with the QFrame
There is a lot of methods which you can use to edit the QFrame. You can check them in QFrame docs. In this tutorial we will only show some of them.

### Checking QFrame size

You can check how many rows returns the query generated by your QFrame by using Python build-in `len()` function.

In [7]:
len(qf)

2

### Doing some basic SQL stuff
Let's now add a `where` statement, rename some fields, add calculated field and remove some fields`.

In [8]:
qf.query("col2 > 1") # <- where

qf.rename({"col1": "items", "col2": "price"})

qf.assign(calculated_field="col4*2", 
          type='num', 
          custom_type='double precision')

qf.remove(["col3", "col4"])

print(qf)

SELECT col1 AS "items",
       col2 AS "price",
       col4*2 AS "calculated_field"
FROM administration.table_tutorial
WHERE col2 > 1


:Be aware that `rename()` method doesn't change the name of the field but only the alias (final name) of the column.:

### Assigning many expressions

You can also assign many expressions at the same time using a dictionary.

In [9]:
new_fields = {f"string_field {i}": f"'{i}'" for i in range(3)}

qf.assign(**new_fields, type="dim", custom_type="VARCHAR(5)")

print(qf)

SELECT col1 AS "items",
       col2 AS "price",
       col4*2 AS "calculated_field",
       '0' AS "string_field 0",
       '1' AS "string_field 1",
       '2' AS "string_field 2"
FROM administration.table_tutorial
WHERE col2 > 1


We created three new fields with spaces using a loop. We will remove these fields as we won't need them in next sections.

In [10]:
qf.remove(new_fields.keys())

### Forking

Forking qframes can be important if your data workflow needs to take the same sql table and apply different transformations to it.

Sometimes we want to fork, do some transforms, then union the QFrames back together which results into an append operation on the data side.

Let's create two copies of one QFrame.

In [11]:
qf1 = qf.copy()
qf2 = qf.copy()

## Unioning data

There are two ways of unioning two QFrames - we can union by the position of the field or by the final name of the columns (that means the alias). 

In [12]:
from grizly import union

qf1.rename({"col2": "price_1", "calculated_field": "price_2"})
qf2.rename({"col2": "price_2", "calculated_field": "price_1"})

#### Union by the positon

In [13]:
uqf_pos = union(qframes=[qf1, qf2], union_type="UNION ALL", union_by='position')
print(uqf_pos)

SELECT col1 AS "items",
       col2 AS "price_1",
       col4*2 AS "price_2"
FROM administration.table_tutorial
WHERE col2 > 1
UNION ALL
SELECT col1 AS "items",
       col2 AS "price_2",
       col4*2 AS "price_1"
FROM administration.table_tutorial
WHERE col2 > 1


#### Union by the column names

In [14]:
uqf_name = union(qframes=[qf1, qf2], union_type="UNION ALL", union_by='name')
print(uqf_name)

SELECT col1 AS "items",
       col2 AS "price_1",
       col4*2 AS "price_2"
FROM administration.table_tutorial
WHERE col2 > 1
UNION ALL
SELECT col1 AS "items",
       col4*2 AS "price_1",
       col2 AS "price_2"
FROM administration.table_tutorial
WHERE col2 > 1


You can see that in this case union changes the order of the columns. 

## Joining data

In [15]:
from grizly import join

We will be using `Chinook.sqlite` to visualize data.

In [16]:
dsn_sqlite = get_path("grizly_dev", "tests", "Chinook.sqlite")

### Simple join

First table is `Track` table.

In [17]:
tracks = {  'select': {
                'fields': {
                    'TrackId': { 'type': 'dim'},
                    'Name': {'type': 'dim'},
                    'AlbumId': {'type': 'dim'},
                    'Composer': {'type': 'dim'},
                    'UnitPrice': {'type': 'num'}
                },
                'table': 'Track'
            }
}
tracks_qf = QFrame(dsn=dsn_sqlite, db="sqlite", dialect="mysql").from_dict(tracks)
print(tracks_qf)

SELECT TrackId,
       Name,
       AlbumId,
       Composer,
       UnitPrice
FROM Track


In [18]:
tracks_qf.to_df().sample(5)

,TrackId,Name,AlbumId,Composer,UnitPrice
3337,3338,The Beginning of the End,261,None,1.99
2517,2518,Pretty Noose,203,Chris Cornell,0.99
1923,1924,Canção Do Novo Mundo,158,"Beto Guedes, Ronaldo Bastos",0.99
3167,3168,The Hard Part,228,None,1.99
1340,1341,Quest For Fire,106,Steve Harris,0.99


The second table is `PlaylistTrack` table. 

In [19]:
playlist_track_qf = QFrame(dsn=dsn_sqlite, db="sqlite", dialect="mysql").from_table(table="PlaylistTrack")

print(playlist_track_qf)

SELECT PlaylistId,
       TrackId
FROM PlaylistTrack


In [20]:
playlist_track_qf.to_df().sample(5)

,PlaylistId,TrackId
2680,1,2183
6124,8,1200
8379,10,2905
1030,1,1095
515,1,3425


Now let's join them on `TrackId`.

In [21]:
joined_qf = join([tracks_qf,playlist_track_qf], 
                 join_type="left join", 
                 on="sq1.TrackId=sq2.TrackId")

print(joined_qf)

SELECT sq1.TrackId AS "TrackId",
       sq1.Name AS "Name",
       sq1.AlbumId AS "AlbumId",
       sq1.Composer AS "Composer",
       sq1.UnitPrice AS "UnitPrice",
       sq2.PlaylistId AS "PlaylistId"
FROM
  (SELECT TrackId,
          Name,
          AlbumId,
          Composer,
          UnitPrice
   FROM Track) sq1
LEFT JOIN
  (SELECT PlaylistId,
          TrackId
   FROM PlaylistTrack) sq2 ON sq1.TrackId=sq2.TrackId


In [22]:
joined_qf.to_df().sample(5)

,TrackId,Name,AlbumId,Composer,UnitPrice,PlaylistId
1521,606,Summertime,48,Miles Davis,0.99,8
1963,810,Hold On,65,D.Coverdal/G.Hughes/Glenn Hughes/I.Paice/Ian P...,0.99,8
2391,960,What A Day,76,"Mike Bordin, Billy Gould, Mike Patton",0.99,1
8126,3314,One For The Road,258,E. Schrody/L. Dimant/L. Muggerud,0.99,8
366,149,Black Sabbath,16,None,0.99,8


As you can see in this example `UnitPrice` is taken from the first table. By default join function is taking all fields from the first QFrame, then all the fields from the second QFrame which are not in the first and so on. If you still want to keep all fields from each QFrame we have to set `unique_col=False`. We will see in the next example how does it work.

### Multiple join

Now let's use one more table to check how does multiple join look like.

In [23]:
playlists_qf = QFrame(dsn=dsn_sqlite, db="sqlite", dialect="mysql").from_table(table="Playlist")

print(playlists_qf)

SELECT PlaylistId,
       Name
FROM Playlist


In [24]:
playlists_qf.to_df().sample(5)

,PlaylistId,Name
0,1,Music
16,17,Heavy Metal Classic
17,18,On-The-Go 1
15,16,Grunge
8,9,Music Videos


Now if we want to join `Tracks`, `PlaylistTrack` and `Playlist` tables we can use `TrackId` and `PlaylistId`. We can see that in `Tracks` and `Playlist` tables we have the same column `Name`. Let's check the option `unique_col=False` and analyse duplicated columns.

In [25]:
joined_qf = join(qframes=[tracks_qf, playlist_track_qf, playlists_qf], 
                 join_type=['left join', 'left join'], 
                 on=['sq1.TrackId=sq2.TrackId', 'sq2.PlaylistId=sq3.PlaylistId'], 
                 unique_col=False)

Please remove or rename duplicated columns. Use your_qframe.show_duplicated_columns() to check duplicates.


In [26]:
joined_qf.show_duplicated_columns()

 DUPLICATED COLUMNS: 
 
TrackId:	 ['sq1.TrackId', 'sq2.TrackId']

Name:	 ['sq1.Name', 'sq3.Name']

PlaylistId:	 ['sq2.PlaylistId', 'sq3.PlaylistId']

Use your_qframe.remove() to remove or your_qframe.rename() to rename columns.


We can see that three columns occure in two different tables. We will remove `sq2.TrackId` and  `sq2.PlaylistId` fields and rename `Name` column.

In [27]:
joined_qf.remove(['sq2.TrackId', 
                  'sq2.PlaylistId']).rename({'sq1.Name': 'TrackName', 
                                             'sq3.Name': 'PlaylistType'})
print(joined_qf)

SELECT sq1.TrackId AS "TrackId",
       sq1.Name AS "TrackName",
       sq1.AlbumId AS "AlbumId",
       sq1.Composer AS "Composer",
       sq1.UnitPrice AS "UnitPrice",
       sq3.PlaylistId AS "PlaylistId",
       sq3.Name AS "PlaylistType"
FROM
  (SELECT TrackId,
          Name,
          AlbumId,
          Composer,
          UnitPrice
   FROM Track) sq1
LEFT JOIN
  (SELECT PlaylistId,
          TrackId
   FROM PlaylistTrack) sq2 ON sq1.TrackId=sq2.TrackId
LEFT JOIN
  (SELECT PlaylistId,
          Name
   FROM Playlist) sq3 ON sq2.PlaylistId=sq3.PlaylistId


In [28]:
joined_qf.to_df().sample(5)

,TrackId,TrackName,AlbumId,Composer,UnitPrice,PlaylistId,PlaylistType
3226,1301,Acacia Avenue,102,None,0.99,1,Music
4892,1994,Spank Thru,163,Kurt Cobain,0.99,5,90’s Music
5661,2305,Binky The Doormat,189,Bill Berry-Peter Buck-Mike Mills-Michael Stipe,0.99,1,Music
2101,860,Eu Te Devoro,69,None,0.99,8,Music
6629,2687,Thief In The Night,217,De Beauport/Jagger/Richards,0.99,8,Music


## Pivot

Again we will use `Chinook.sqlite` and `Track` table to visualize data.

In [29]:
qf = QFrame(dsn=dsn_sqlite, db="sqlite", dialect="mysql").from_table(table="Track")

len(qf)

3503

Our table has `3503` rows - we will limit the data to `15` rows to get better view. We will use `QFrame.window()` method to be sure that the result is deterministic. 

In [30]:
qf.window(offset=90, limit=15, order_by=["TrackId"])
qf.to_df()

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,91,Shadow on the Sun,10,1,1,Audioslave/Chris Cornell,343457,8245793,0.99
1,92,I am the Highway,10,1,1,Audioslave/Chris Cornell,334942,8041411,0.99
2,93,Exploder,10,1,1,Audioslave/Chris Cornell,206053,4948095,0.99
3,94,Hypnotize,10,1,1,Audioslave/Chris Cornell,206628,4961887,0.99
4,95,Bring'em Back Alive,10,1,1,Audioslave/Chris Cornell,329534,7911634,0.99
5,96,Light My Way,10,1,1,Audioslave/Chris Cornell,303595,7289084,0.99
6,97,Getaway Car,10,1,1,Audioslave/Chris Cornell,299598,7193162,0.99
7,98,The Last Remaining Light,10,1,1,Audioslave/Chris Cornell,317492,7622615,0.99
8,99,Your Time Has Come,11,1,4,"Cornell, Commerford, Morello, Wilk",255529,8273592,0.99
9,100,Out Of Exile,11,1,4,"Cornell, Commerford, Morello, Wilk",291291,9506571,0.99


In [31]:
qf.pivot(rows=["Composer"], columns=["AlbumId", "GenreId"], values="UnitPrice", aggtype="sum")
qf.to_df()

,Composer,10_1,11_4
0,Audioslave/Chris Cornell,7.92,0.00
1,"Cornell, Commerford, Morello, Wilk",0.00,6.93


As you can see all values in `AlbumId` and `GenreId` became separate columns, `Composer` column has been grouped and `UnitPrice` has been sumed up.